In [ ]:
!pip install psycopg2

In [1]:
import os
import cx_Oracle
import pandas as pd

import webbrowser
import psycopg2

In [2]:
#config file
redshift_conn_params = {'database':'dev', 'host': 'testing-fw-wgrp.419835568062.ap-south-1.redshift-serverless.amazonaws.com', 'port': 5439, 
                  'username': 'admin', 'password': 'Saama123'}

oracle_conn_params = {'service_name' :"database", 'host': 'oracle-db.c2jpp1uei2md.ap-south-1.rds.amazonaws.com', 'port': 1521,
                  'username': 'admin', 'password': 'Saama123'}

table_name = 'customers'





#=======================

ora_user = "admin"
ora_pwd = "Saama123"
host = "oracle-db.c2jpp1uei2md.ap-south-1.rds.amazonaws.com"
port = 1521
service_name = "DATABASE"
ora_tab_name = "CUSTOMERS"


# Connection parameters for redshift

redshift_db = 'dev'
redshift_user = 'admin'
redshift_pwd = 'Saama123'
redshift_host = 'testing-fw-wgrp.419835568062.ap-south-1.redshift-serverless.amazonaws.com'
redshift_port = '5439'
redshift_tab_name = 'customers'

In [3]:
cx_Oracle.init_oracle_client(
    lib_dir=r"C:\Users\kanch\Downloads\instantclient-basic-windows.x64-21.10.0.0.0dbru\instantclient_21_10")

In [4]:
def get_oracle_columns(user, password, host, port, service_name, table_name):
    """
    Connects to Oracle database and retrieves the columns of a specific table.

    Args:
        user (str): Oracle database username.
        password (str): Oracle database password.
        host (str): Oracle database host.
        port (int): Oracle database port.
        service_name (str): Oracle database service name.
        table_name (str): Name of the table to retrieve columns from.

    Returns:
        list: List of Oracle column descriptions.

    """
    oracle_connection = cx_Oracle.connect(f"{user}/{password}@{host}:{port}/{service_name}")
    oracle_cursor = oracle_connection.cursor()
    oracle_cursor.execute(f"SELECT * FROM {table_name} WHERE 1=0")
    oracle_columns = oracle_cursor.description
    oracle_cursor.close()
    oracle_connection.close()
    return oracle_columns

In [19]:
# Connection redshift 

def get_redshift_table_schema(host, port, database, username, password, table_name):
    # Create a connection string
    conn_string = f"dbname={database} user={username} password={password} host={host} port={port}"
    # Establish a connection
    conn = psycopg2.connect(conn_string)
    # Get a cursor
    cursor = conn.cursor()

    # Get the column data types and field names
    query = f"SELECT column_name, data_type FROM information_schema.columns WHERE table_name = '{table_name}'"
    cursor.execute(query)
    # Fetch the results
    results = cursor.fetchall()

    # Close the cursor and connection
    cursor.close()
    conn.close()

    # Convert the results to a dictionary
    schema_dict = {field: data_type for field, data_type in results}   
   
    return schema_dict

In [61]:
def redshift_to_oracle_datatypes(redshift_field_datatypes):
    
    redshift_to_oracle_mapping = {
        'integer': 'number',
        'bigint': 'number',
        'smallint': 'number',
        'decimal': 'number',
        'numeric': 'number',
        'real': 'float',
        'double precision': 'float',
        'char': 'char',
        'character': 'char',
        'varchar': 'varchar2',
        'character varying': 'varchar2',
        'date': 'date',
        'timestamp': 'timestamp',
        'boolean': 'number'
    }
    converted_datatypes = {}
    for redshift_datatype in redshift_field_datatypes:
        print(redshift_datatype.lower())
        oracle_datatype = redshift_to_oracle_mapping.get(redshift_field_datatypes[redshift_datatype.lower()])
        converted_datatypes[redshift_datatype] = oracle_datatype
    
    return converted_datatypes
    
#ch_res=redshift_to_oracle_datatypes({'full_name': 'character varying', 'email_address': 'character varying', 'customer_id': 'integer'})
 
#print(ch_res)

In [7]:
def get_oracle_columns_and_datatypes(user, password, host, port, service_name, table_name):
    """
    Connects to Oracle database and retrieves the columns of a specific table.

    Args:
        user (str): Oracle database username.
        password (str): Oracle database password.
        host (str): Oracle database host.
        port (int): Oracle database port.
        service_name (str): Oracle database service name.
        table_name (str): Name of the table to retrieve columns from.

    Returns:
        list: List of Oracle column descriptions.

    """
    oracle_connection = cx_Oracle.connect(f"{user}/{password}@{host}:{port}/{service_name}")
    oracle_cursor = oracle_connection.cursor()
    oracle_cursor.execute(f"SELECT * FROM {table_name} WHERE 1=0")
    oracle_columns = oracle_cursor.description
    oracle_cursor.close()
    oracle_connection.close()
    return oracle_columns

In [8]:
def oracle_to_redshift_datatype(oracle_datatype):
    datatype_mapping = {
        "<cx_Oracle.DbType DB_TYPE_BFILE>": "bfile",
        "<cx_Oracle.DbType DB_TYPE_BINARY_DOUBLE>": "binary_double",
        "<cx_Oracle.DbType DB_TYPE_BINARY_FLOAT>": "binary_float",
        "<cx_Oracle.DbType DB_TYPE_BLOB>": "blob",
        "<cx_Oracle.DbType DB_TYPE_CHAR>": "char",
        "<cx_Oracle.DbType DB_TYPE_CLOB>": "clob",
        "<cx_Oracle.DbType DB_TYPE_CURSOR>": "cursor",
        "<cx_Oracle.DbType DB_TYPE_DATE>": "date",
        "<cx_Oracle.DbType DB_TYPE_INTERVAL_DS>": "interval_ds",
        "<cx_Oracle.DbType DB_TYPE_JSON>": "json",
        "<cx_Oracle.DbType DB_TYPE_LONG>": "long",
        "<cx_Oracle.DbType DB_TYPE_LONG_RAW>": "long_raw",
        "<cx_Oracle.DbType DB_TYPE_NCHAR>": "nchar",
        "<cx_Oracle.DbType DB_TYPE_NCLOB>": "nclob",
        "<cx_Oracle.DbType DB_TYPE_NUMBER>": "number",
        "<cx_Oracle.DbType DB_TYPE_NVARCHAR>": "nvarchar",
        "<cx_Oracle.DbType DB_TYPE_OBJECT>": "object",
        "<cx_Oracle.DbType DB_TYPE_RAW>": "raw",
        "<cx_Oracle.DbType DB_TYPE_ROWID>": "rowid",
        "<cx_Oracle.DbType DB_TYPE_TIMESTAMP>": "timestamp",
        "<cx_Oracle.DbType DB_TYPE_TIMESTAMP_LTZ>": "timestamp_ltz",
        "<cx_Oracle.DbType DB_TYPE_VARCHAR>": "varchar2"
        # Add more mappings as needed
    }
    return datatype_mapping.get(oracle_datatype, 'unknown')

In [9]:

def convert_oracle_output_to_dict(oracle_output):
    converted_dict = {}
    for column_info in oracle_output:
        column_name = column_info[0]
        oracle_datatype = str(column_info[1])
        redshift_datatype = oracle_to_redshift_datatype(oracle_datatype)
        converted_dict[column_name.lower()] = redshift_datatype
    return converted_dict

In [59]:

def compare_oracle_redshift_datatypes(redshift_datatypes, oracle_output):
    # Convert Redshift datatypes to lowercase for comparison
    redshift_datatypes_lower = {col.lower(): dtype.lower() for col, dtype in redshift_datatypes.items()}

    # Convert Oracle output to lowercase dictionary
    #oracle_dict = convert_oracle_output_to_dict(oracle_output)
    oracle_dict = oracle_output

    comparison_result = []
    for col, oracle_dtype in oracle_dict.items():
        redshift_dtype = redshift_datatypes_lower.get(col, None)
        match = oracle_dtype == redshift_dtype
        comparison_result.append((col, oracle_dtype, redshift_dtype, match))

    df = pd.DataFrame(comparison_result, columns=['column_name', 'oracle_dtype', 'redshift_dtype', 'match'])
    return df

In [63]:
def generate_excel_report(comparison_results_df, filename):
    # Create a Pandas Excel writer using the filename
    writer = pd.ExcelWriter(filename, engine='xlsxwriter')

    # Write the DataFrame to the Excel file
    comparison_results_df.to_excel(writer, index=True)

    # Save the Excel file
    writer.save()
    print(f"Report has been generated at {filename}")

In [75]:
def generate_csv_report(comparison_results_df, file_path):
    """
    Write a pandas DataFrame to a CSV file.

    Args:
        dataframe (pd.DataFrame): The DataFrame to be written to CSV.
        file_path (str): The file path along with the filename where the CSV file should be saved.
    """
    comparison_results_df.to_csv(file_path, index=False)
    print(f"Report has been generated at {file_path}")

In [62]:
rtdaya=get_redshift_table_schema(redshift_host, redshift_port, redshift_db, redshift_user, redshift_pwd, redshift_tab_name)

redshift_datatypes = redshift_to_oracle_datatypes(
get_redshift_table_schema(redshift_host, redshift_port, redshift_db, redshift_user, redshift_pwd, redshift_tab_name))


orac_dt=get_oracle_columns_and_datatypes(ora_user, ora_pwd, host, port, service_name, ora_tab_name)
oracle_datatypes = convert_oracle_output_to_dict(
get_oracle_columns_and_datatypes(ora_user, ora_pwd, host, port, service_name, ora_tab_name))
print(redshift_datatypes)
print(oracle_datatypes)
df = compare_oracle_redshift_datatypes(redshift_datatypes, oracle_datatypes)
print(df)

'''print(orac_dt)
print(oracle_datatypes)'''


full_name
email_address
customer_id
{'full_name': 'varchar2', 'email_address': 'varchar2', 'customer_id': 'number'}
{'customer_key': 'number', 'email_address': 'varchar2', 'full_name': 'varchar2'}
     column_name oracle_dtype redshift_dtype  match
0   customer_key       number           None  False
1  email_address     varchar2       varchar2   True
2      full_name     varchar2       varchar2   True


'print(orac_dt)\nprint(oracle_datatypes)'

In [76]:
generate_csv_report(df, 'C:/Users/kanch/data_diff.csv')

Report has been generated at C:/Users/kanch/data_diff.csv


ValueError: Invalid extension for engine '<property object at 0x000001D734F94090>': 'csv'